## Parse Yosys JSON netlist

In [207]:
import json

In [208]:
def elaborate(top):
    cells = top['cells']
    for key in cells.keys():
        cell = cells[key]
        type = cell['type']

        # see if the module is a 74-series chip. If we did, stop (we don't care what's inside the chip/how it's modeled)
        if type.startswith('\\74'):
            print(type, cell)

        # Otherwise, we need to see if there are 74-series chips inside the module
        else:
            mod = modules[type]
            if 'cells' in mod.keys():
                elaborate(modules[type])

In [209]:
with open('../out.json') as fp:
    js = json.load(fp)

modules = js['modules']

top = modules['dumb_op_card']

# The card IO wires
print('Ports:', top['ports'])
print()

# The card chips
for c in top['cells'].keys():
    type = top['cells'][c]['type']
    if not type.startswith('\\74') or type.startswith('\\74AC'):
        print('Error: ' + type + ' is not a 74-series chip! Skipping logic implementation\n')
        continue
    print(type + '\n', top['cells'][c]['connections'], sep='\t')


Ports: {'clk': {'direction': 'input', 'bits': [2]}, '~rst': {'direction': 'input', 'bits': [3]}, '~int': {'direction': 'input', 'bits': [4]}, 'VCC_5V': {'direction': 'input', 'bits': [5]}, 'GND': {'direction': 'input', 'bits': [6]}, 'VCC_3V3': {'direction': 'input', 'bits': [7]}, '~we': {'direction': 'input', 'bits': [8]}, '~oe': {'direction': 'input', 'bits': [9]}, 'addr': {'direction': 'input', 'bits': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]}, 'aux-bus': {'direction': 'input', 'bits': [26, 27, 28, 29, 30, 31, 32, 33]}, 'data': {'direction': 'output', 'bits': [34, 35, 36, 37, 34, 35, 36, 37]}}

Error: \74AC04_6x1NOT is not a 74-series chip! Skipping logic implementation

\7404
	{'\\1A': [10], '\\1Y': [39], '\\2A': [11], '\\2Y': [40], '\\3A': [12], '\\3Y': [41], '\\4A': [13], '\\4Y': [42], '\\5A': ['0'], '\\6A': ['0'], 'GND': ['0'], 'VCC': ['1']}
\74161
	{'A': [39], 'B': [40], 'C': [41], 'CLK': [38], 'D': [42], 'ENP': ['1'], 'ENT': ['1'], 'GND': ['0'], 'QA': [3

In [230]:
import glob
import os
paths = glob.glob('../74xx/**/**.dig')
paths

import xml.etree.ElementTree as ET

pinouts = {}
for path in paths:
    chip_name = os.path.basename(path)
    chip_name = chip_name[:chip_name.find('.dig')]
    p = ET.parse(path).getroot()
    pinout = {}
    visualElements = None # will contain all pins
    for child in p:
        if child.tag == 'visualElements':
            visualElements = child
            break

    for ve_list in visualElements:
        valid = False
        pin_name = None
        pin_num = None
        for ve_list_item in ve_list:
            if ve_list_item.tag == 'elementName':
                t = ve_list_item.text
                if t == 'In' or t == 'Out':
                    valid = True
            if ve_list_item.tag == 'elementAttributes':
                for a in ve_list_item:
                    check = False
                    label = None
                    for b in a:
                        if b.tag == 'string':
                            if not b.text == 'Label':
                                label = b.text
                            else:
                                check = True
                    if check:
                        pin_name = label

                for a in ve_list_item:
                    check1 = False
                    num = None
                    for b in a:
                        if b.tag == 'string':
                            if not b.text == 'pinNumber':
                                num = b.text
                            else:
                                check1 = True
                    if check1:
                        pinout[pin_name] = num
    pinouts[chip_name] = pinout
    print(chip_name, '    \t', pinout)

74163     	 {'~PE': '9', '~SR': '1', 'CP': '2', 'TC': '15', 'CET': '10', 'CEP': '7', 'P0': '3', 'P1': '4', 'P2': '5', 'P3': '6', 'Q0': '14', 'Q1': '13', 'Q2': '12', 'Q3': '11', 'VCC': '16', 'GND': '8'}
74191     	 {'Q0': '3', 'Q1': '2', 'Q2': '6', 'Q3': '7', 'TC': '12', 'D0': '15', '~PL': '11', '~U/D': '5', 'CP': '14', '~CE': '4', '~RC': '13', 'D1': '1', 'D2': '10', 'D3': '9', 'VCC': '16', 'GND': '8'}
74779     	 {'CP': '15', 'S0': '11', 'S1': '10', '~OE': '9', 'I/O0': '16', 'I/O1': '1', 'I/O2': '2', 'I/O3': '3', 'I/O4': '5', 'I/O5': '6', 'I/O6': '7', 'I/O7': '8', '~CET': '14', '~TC': '12', 'VCC': '13', 'GND': '4'}
74190     	 {'Q0': '3', 'Q1': '2', 'Q2': '6', 'Q3': '7', 'TC': '12', 'D0': '15', '~PL': '11', '~U/D': '5', 'CP': '14', '~CE': '4', '~RC': '13', 'D1': '1', 'D2': '10', 'D3': '9', 'VCC': '16', 'GND': '8'}
74162     	 {'~LD': '9', '~CLR': '1', 'CLK': '2', 'RCO': '15', 'ENT': '10', 'ENP': '7', 'A': '3', 'QA': '14', 'QB': '13', 'QC': '12', 'QD': '11', 'B': '4', 'C': '5', 'D': '6'